# SciPy Linear Algebra Tutorial

A practical tour of `scipy.linalg` (dense) and `scipy.sparse.linalg` (sparse),
with examples, tips, and exercises.

**Contents**
1. Setup & imports
2. Dense linear algebra basics (`scipy.linalg`)
3. Matrix factorizations (LU / QR / Cholesky / SVD)
4. Eigenvalues & eigenvectors (`eig`, `eigh`)
5. Least squares & conditioning
6. Matrix functions (`expm`, `logm`, `sqrtm`)
7. Sparse matrices & solvers (`spsolve`, `cg`, `gmres`)
8. Practical tips & pitfalls
9. Exercises


## 1) Setup & imports

In [ ]:
import numpy as np
from numpy.linalg import norm as np_norm
import scipy
from scipy import linalg as LA
from scipy.sparse import csr_matrix, diags, random as sparse_random
from scipy.sparse.linalg import spsolve, cg, gmres, eigs, eigsh
scipy.__version__

## 2) Dense linear algebra basics (`scipy.linalg`)
We'll solve linear systems, compute determinants, inverses (and learn when **not** to),
and use norms & condition numbers.

In [ ]:
# Toy system Ax = b
A = np.array([[3., 2., -1.],
              [2., -2., 4.],
              [-1., 0.5, -1.]])
b = np.array([1., -2., 0.])

# Solve Ax=b
x = LA.solve(A, b)
x, np.allclose(A @ x, b)

In [ ]:
# Determinant, inverse (avoid in practice), condition number, norms
detA = LA.det(A)
A_inv = LA.inv(A)  # for teaching/demo; prefer solve()
condA = LA.cond(A)
two_norm = LA.norm(A, 2)
fro_norm = LA.norm(A, 'fro')
detA, condA, two_norm, fro_norm

## 3) Matrix factorizations
Factorizations give numerically stable, reusable pieces to solve systems efficiently.

### 3.1 LU (for general square systems)

In [ ]:
P, L, U = LA.lu(A)
P, L, U, np.allclose(P @ A, L @ U)

In [ ]:
# Using LU factor/solve for repeated right-hand sides
lu, piv = LA.lu_factor(A)
b1 = np.array([1., 0., 0.])
b2 = np.array([0., 1., 0.])
x1 = LA.lu_solve((lu, piv), b1)
x2 = LA.lu_solve((lu, piv), b2)
np.allclose(A @ x1, b1), np.allclose(A @ x2, b2)

### 3.2 QR (least squares, orthogonalization)

In [ ]:
M = np.array([[1., 1.], [1., -1.], [1., 2.]])  # 3x2
y = np.array([2., 0., 3.])
Q, R = LA.qr(M, mode='economic')
beta = LA.solve_triangular(R, Q.T @ y)
beta

### 3.3 Cholesky (SPD matrices)

In [ ]:
S = np.array([[4., 2.], [2., 3.]])  # symmetric positive definite
L = LA.cholesky(S, lower=True)
x = LA.cho_solve((L, True), np.array([1., 0.]))
L, x, np.allclose(S @ x, np.array([1., 0.]))

### 3.4 SVD (low-rank structure, pseudoinverse, PCA)

In [ ]:
A2 = np.array([[1., 0., 0.], [0., 2., 0.], [0., 0., 0.5]]) @ np.random.randn(3, 5)
U, s, Vh = LA.svd(A2, full_matrices=False)
rank = np.sum(s > 1e-12)
pinv = LA.pinv(A2)
s, rank, pinv.shape

## 4) Eigenvalues & eigenvectors (`eig`, `eigh`)

In [ ]:
W, V = LA.eig(A)        # general (possibly non-symmetric)
Ws, Vs = LA.eigh(S)      # symmetric/Hermitian
W, Ws

## 5) Least squares & conditioning

In [ ]:
beta2, residuals, rnk, svals = LA.lstsq(M, y)  # via SVD under the hood
condM = LA.cond(M)
beta2, residuals, rnk, condM

## 6) Matrix functions (`expm`, `logm`, `sqrtm`)
Useful in differential equations, diffusion processes, and control.

In [ ]:
from scipy.linalg import expm, logm, sqrtm
B = np.array([[0., 1.], [-2., -3.]])
eB = expm(B)
sB = sqrtm(S)
# logm requires matrices with appropriate properties
eB, sB

## 7) Sparse matrices & solvers (`spsolve`, `cg`, `gmres`)

In [ ]:
# Create a sparse SPD system (1D Laplacian)
n = 10
main = 2*np.ones(n)
off = -1*np.ones(n-1)
L1d = diags([off, main, off], [-1, 0, 1], format='csr')
rhs = np.ones(n)

x_spsolve = spsolve(L1d, rhs)
x_cg, info_cg = cg(L1d, rhs, tol=1e-10)
x_gm, info_gm = gmres(L1d, rhs, tol=1e-10)
x_spsolve[:5], info_cg, info_gm

In [ ]:
# Sparse eigen (largest magnitude / largest algebraic)
k = 3
vals, vecs = eigs(L1d, k=k)     # general sparse
vals_sym, vecs_sym = eigsh(L1d, k=k)  # symmetric/Hermitian
vals, vals_sym

## 8) Practical tips & pitfalls
- Prefer `solve` over forming explicit inverses.
- Use problem structure: `cho_solve` for SPD, QR/SVD for least squares.
- Check condition numbers; rescale or regularize ill-conditioned problems.
- For large/sparse problems, prefer `spsolve`, `cg`, `gmres`, and preconditioning.
- Use appropriate dtypes (`float64`) for numerical stability.


## 9) Exercises
1. **Repeated RHS**: Factorize a random 200×200 matrix with LU and solve for 3 different right-hand sides.
2. **Cholesky vs LU**: Generate SPD matrices and compare solve performance/accuracy using `cho_solve` vs `lu_solve`.
3. **Low-rank approximation**: For a random matrix, compute SVD and form the best rank-1 approximation. Measure the Frobenius error.
4. **Least squares stability**: Compare solutions from normal equations (`(X^T X)^{-1} X^T y`) vs `lstsq` on a badly conditioned design matrix.
5. **Sparse Poisson**: Build a 2D Laplacian with Kronecker sums, solve with `cg`, and experiment with different tolerances.
